<a href="https://colab.research.google.com/github/kashishthakur26/Natural_language_processing/blob/main/Multi_Label_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Label Text Classification for Stack Overflow Tag Prediction

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/stackoverflow.csv', index_col = 0)

In [4]:
df.head()

,Text,Tags
2,aspnet site maps has anyone got experience cre...,"['sql', 'asp.net']"
4,adding scripting functionality to net applicat...,"['c#', '.net']"
5,should i use nested classes in this case i am ...,['c++']
6,homegrown consumption of web services i have b...,['.net']
8,automatically update version number i would li...,['c#']


In [5]:
type(df.iloc[0]['Tags'])

str

In [6]:
import ast

In [7]:
df['Tags'] = df['Tags'].apply(lambda x: ast.literal_eval(x))

In [8]:
df.head()

,Text,Tags
2,aspnet site maps has anyone got experience cre...,"[sql, asp.net]"
4,adding scripting functionality to net applicat...,"[c#, .net]"
5,should i use nested classes in this case i am ...,[c++]
6,homegrown consumption of web services i have b...,[.net]
8,automatically update version number i would li...,[c#]


In [9]:
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(df['Tags'])

In [10]:
classes = multilabel.classes_

In [11]:
classes

array(['.net', 'android', 'asp.net', 'c', 'c#', 'c++', 'css', 'html',
       'ios', 'iphone', 'java', 'javascript', 'jquery', 'mysql',
       'objective-c', 'php', 'python', 'ruby', 'ruby-on-rails', 'sql'],
      dtype=object)

In [12]:
pd.DataFrame(y, columns=classes)

,.net,android,asp.net,c,c#,c++,css,html,ios,iphone,java,javascript,jquery,mysql,objective-c,php,python,ruby,ruby-on-rails,sql
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48971,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48972,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48973,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
48974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [13]:
tfidf = TfidfVectorizer(analyzer='word', max_features = 1000, ngram_range = (1,3), stop_words='english')

In [14]:
X = tfidf.fit_transform(df['Text'])

In [15]:
X

<48976x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1239765 stored elements in Compressed Sparse Row format>

In [16]:
X.shape, y.shape

((48976, 1000), (48976, 20))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [18]:
X_train.shape, X_test.shape

((39180, 1000), (9796, 1000))

## Model Building

In [19]:
from sklearn.multiclass import OneVsRestClassifier

In [20]:
def j_score(y_true, y_pred):
    jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
    return jaccard.mean()*100

In [21]:
lr = LogisticRegression(solver='lbfgs')

In [22]:
clf = OneVsRestClassifier(lr)
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [23]:
y_pred = clf.predict(X_test)

In [25]:
j_score(y_test, y_pred)

49.12668436096366

## SVM

In [24]:
from sklearn.svm import LinearSVC

In [26]:
svm = LinearSVC(C=1.5  ,penalty='l1',dual=False)
clf = OneVsRestClassifier(svm)
clf.fit(X_train , y_train)

OneVsRestClassifier(estimator=LinearSVC(C=1.5, dual=False, penalty='l1'))

In [27]:
y_pred = clf.predict(X_test)
j_score(y_test, y_pred)

53.3787600381108

In [28]:
x= ["how to write ml code in python and java i have data but don not know how to do it"]

In [29]:
xt = tfidf.transform(x)

In [30]:
clf.predict(xt)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

In [31]:
multilabel.inverse_transform(clf.predict(xt))

[('java', 'python')]